### **0. Instalacion e imports de librerias**

In [84]:
import sys
# Version 3.8 de Python requerida
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install contexto
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install openpyxl

In [85]:
from contexto.limpieza import limpieza_texto, lista_stopwords, remover_stopwords
from contexto.stemming import Stemmer, stem_texto
from contexto.vectorizacion import VectorizadorFrecuencias

import pandas as pd

### **1. Limpieza de datos**

In [86]:
df = pd.read_excel('cat_6716.xlsx')

In [87]:
df.head()

,Textos_espanol,sdg
0,"Es importante destacar que, en un año de sequí...",6
1,Hay una gran cantidad de literatura sobre Aust...,6
2,"Los procesos de descentralización, emprendidos...",6
3,Esto puede tener consecuencias sustanciales pa...,6
4,La función de beneficio también incorpora pará...,6


In [88]:
# Pasar todo el texto a minúsculas
# Quitar signos de puntuación
# Quitar stopwords (palabras y/o expresiones): Por el momento no
# Quitar números (configurable): True
# Quitar acentos (configurable). True

df['Textos_espanol'] = df['Textos_espanol'].apply(lambda x: limpieza_texto(texto=x, quitar_numeros=True, quitar_acentos=True))
df.head()

,Textos_espanol,sdg
0,es importante destacar que en un ano de sequia...,6
1,hay una gran cantidad de literatura sobre aust...,6
2,los procesos de descentralizacion emprendidos ...,6
3,esto puede tener consecuencias sustanciales pa...,6
4,la funcion de beneficio tambien incorpora para...,6


In [89]:
print(df['Textos_espanol'][0])

es importante destacar que en un ano de sequia se espera que disminuyan todos los aportes pero que aumente la demanda de agua de los cultivos y de la vegetacion lo que provocara una reduccion del caudal de los rios que estan conectados hidrologicamente con el acuifero asi como una menor disponibilidad de agua para otros ecosistemas dependientes de las aguas subterraneas estas externalidades espaciales pueden incluir el agotamiento de los cursos de agua y el descenso del nivel freatico local por ejemplo el bombeo continuo del pozo b provocara el agotamiento de los cursos de agua y la reduccion de su caudal


In [90]:
stopwords = lista_stopwords()

def remove_stopwords_from_text(text):
    return remover_stopwords(text, stopwords)

df['Textos_espanol'] = df['Textos_espanol'].apply(lambda x: remove_stopwords_from_text(x))

df.head()

,Textos_espanol,sdg
0,importante destacar ano sequia espera disminuy...,6
1,gran cantidad literatura australia area sugier...,6
2,procesos descentralizacion emprendidos serie a...,6
3,puede tener consecuencias sustanciales calidad...,6
4,funcion beneficio tambien incorpora parametros...,6


In [91]:
print(df['Textos_espanol'][0])

importante destacar ano sequia espera disminuyan aportes aumente demanda agua cultivos vegetacion provocara reduccion caudal rios estan conectados hidrologicamente acuifero menor disponibilidad agua ecosistemas dependientes aguas subterraneas externalidades espaciales pueden incluir agotamiento cursos agua descenso nivel freatico local ejemplo bombeo continuo pozo provocara agotamiento cursos agua reduccion caudal


### **2. Stemming**

In [92]:
stemmer = Stemmer(lenguaje='español')

In [93]:
df.head()

df['Textos_espanol'] = df['Textos_espanol'].apply(lambda x: stem_texto(texto=x, stemmer=stemmer))
df.head()

,Textos_espanol,sdg
0,import destac ano sequi esper disminu aport au...,6
1,gran cantid literatur australi are sugier fuer...,6
2,proces descentralizacion emprend seri activ re...,6
3,pued ten consecuent sustancial calid agu espec...,6
4,funcion benefici tambi incorpor parametr afect...,6


### **3. Vectorizacion**

In [94]:
# Vectorizador BOW
v_bow = VectorizadorFrecuencias()

In [95]:
# Hacer una lista, donde cada elemento es un string de la posicion Textos_espanol

textos = [x for x in df['Textos_espanol']]

v_bow.ajustar(textos, archivo_salida='vocabulario.txt')

In [96]:
from IPython.display import display

display(v_bow.vocabulario().head(100))

,posición,palabra
0,0,aa
1,1,aan
2,2,aarhus
3,3,aas
4,4,abaj
...,...,...
95,95,acidificacion
96,96,aclam
97,97,aclar
98,98,aclaracion


In [107]:
df['Vector'] = df['Textos_espanol'].apply(lambda x: v_bow.vectorizar(textos=x, disperso=False))
df.head()

,Textos_espanol,sdg,Vector
0,import destac ano sequi esper disminu aport au...,6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,gran cantid literatur australi are sugier fuer...,6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,proces descentralizacion emprend seri activ re...,6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,pued ten consecuent sustancial calid agu espec...,6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,funcion benefici tambi incorpor parametr afect...,6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [112]:
print(df['Textos_espanol'][0])

import destac ano sequi esper disminu aport aument demand agu cultiv vegetacion provoc reduccion caudal rios estan conect hidrolog acuifer menor disponibil agu ecosistem dependient agu subterran external espacial pued inclu agot curs agu descens nivel freatic local ejempl bombe continu poz provoc agot curs agu reduccion caudal


In [111]:
v_bow.inversa(df['Vector'][0])

[array(['acuifer', 'agot', 'agu', 'ano', 'aport', 'aument', 'bombe',
        'caudal', 'conect', 'continu', 'cultiv', 'curs', 'demand',
        'dependient', 'descens', 'destac', 'disminu', 'disponibil',
        'ecosistem', 'ejempl', 'espacial', 'esper', 'estan', 'external',
        'freatic', 'hidrolog', 'import', 'inclu', 'local', 'menor',
        'nivel', 'poz', 'provoc', 'pued', 'reduccion', 'rios', 'sequi',
        'subterran', 'vegetacion'], dtype='<U24')]